## Model deployment

Let's deploy our model into the Watson Machine Learning (WML) repository. For the deployment we'll use the model that uses both air quality indes and weather data i.e. the model that performed the best with the validation set.

In [1]:
from keras.models import load_model
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Using TensorFlow backend.
/Users/aapo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/aapo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/aapo/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/aapo/.local/lib/python3.7/site-packages/tensorflow/python/fra

In [5]:
!tar -zcvf models/air-quality-index.tgz models/lstm-weather-data-look-back-1.h5

a models/lstm-weather-data-look-back-1.h5


In [7]:
import keras
print('Current:\t', keras.__version__)

Current:	 2.2.4


In [8]:
import tensorflow as tf
print('Current:\t', tf.__version__)


Current:	 1.13.1


In [11]:
wml_credentials={
  "apikey": "tcA_e-Shzd5Ajlz8nDqFvlI9Z6woNhvKZRSo--0nWs02",
  "iam_apikey_description": "Auto-generated for key 3ce409c1-e165-4419-9944-47b5488ae937",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/5575a60546e94de1ad43d59e8f6fcd25::serviceid:ServiceId-9bbef2c3-b976-4fe9-85b2-e3ec53e0af03",
  "instance_id": "120aed25-4411-4dae-a902-e224b20bf1ec",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [32]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [33]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "ARa", 
               client.repository.ModelMetaNames.NAME: "ADS_Capstone_AirQualityIndex",
               client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
               client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.13" ,
               client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{"name": "keras", "version": "2.2.4"}]
              }



In [44]:
published_model = client.repository.store_model(model="cos://eu-de/air-quality-index-model-bucket/air-quality-index.tgz", meta_props=model_props)

2019-08-02 14:05:08,718 - watson_machine_learning_client.wml_client_error - WARNING - Invalid path: neither file nor directory exists under this path: 'cos://eu-de/air-quality-index-model-bucket/air-quality-index.tgz'.


WMLClientError: Invalid path: neither file nor directory exists under this path: 'cos://eu-de/air-quality-index-model-bucket/air-quality-index.tgz'.

In [36]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [45]:
client.deployments.list()

----  ----  ----  -----  -------  ---------  -------------
GUID  NAME  TYPE  STATE  CREATED  FRAMEWORK  ARTIFACT TYPE
----  ----  ----  -----  -------  ---------  -------------


In [42]:
#client.deployments.delete("92d0dd61-cc23-486f-954c-cceae5a036a7")
#client.deployments.delete("5b689870-0b12-44b0-94db-eae55040fd34")


created_deployment = client.deployments.create(published_model_uid, name="ADS_Capstone_AirQualityIndex")

'SUCCESS'